# Data Wrangling

Always start with importing the required imports

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import math
import yfinance as yf
from datetime import datetime, timedelta
import time
import io

************

## Voting Data

### govtrack.us


From this website, I am going to aquire the dataset of the legislative bills for the house and senate that were voted on between 2019 and 2022. I chose this timeframe because the politician trade data only goes back to the end of 2019, and I figured going back prior to 2019 would not give much insight to trades made over a year later.

I went to the govtrack.us website to manually determine how many legislations there were in each year. From there, I would scrape the website to download the CSVs that exist on each legislation's page.

Before scraping, I decided to set up the dataframes:

The scraping requires 3 variables for each get request:

'https://www.govtrack.us/congress/votes/{congress}-{year}/{h}/export/csv'

They are the year, the congress number, and the legislation number per chamber. For example, "2019" would be the congress number "116", and a single legislation would be "h5". 
I decided to name each legislation by "chamber initial" "legislation number of that year"-"year". For example, "h5-2021" would be the house's 5th legislation of the year 2021.

First, create the house and senate list ranges

In [13]:
# number of legislations for each chamber for each year
chamber_count = {}

# quantities taken from the website
chamber_count[2019] =[701,428] 
chamber_count[2020] =[253,292]
chamber_count[2021] =[449,528] 
chamber_count[2022] =[468,357] 

Then, create a house dictionary and a senate dictionary, each to hold a list of legislations per year.

In [14]:
# create a dictionary for each chamber per year
house = {}
senate = {}
years = [2019,2020,2021,2022]

for year in years:
    house[year] =[]
    senate[year] = []
    
    # per legislation per year, create an associated "column header" and append to a list
    # for the house
    for h in range(1,chamber_count[year][0]+1):
        house[year].append(f'h{h}')
    # for the senate
    for s in range(1,chamber_count[year][1]+1):    
        senate[year].append(f's{s}')

Sanity check to make sure the length of each list matches the number of legislations I recorded above.

In [15]:
print('House:',len(house[2019]),len(house[2020]),len(house[2021]),len(house[2022]))
print('Senate:',len(senate[2019]),len(senate[2020]),len(senate[2021]),len(senate[2022]))

House: 701 253 449 468
Senate: 428 292 528 357


All good!

Now, I'm going to import each of the house CSVs if they exist. If any do not exist, I will delete it from the list.

In [16]:
# for year in years:
    # in 2019 and 2020, it was the 116th congress. In 2021 and 2022, it was the 117the congress.
    if year < 2021: congress = 116
    else: congress = 117
        
    # create list of legislations to drop if they do not exist
    h_drop = []

    # Scraping   
    for h in house[year]:
        # first make sure the request is valid. If it is not, drop the legislation from the list.
        if requests.get(f'https://www.govtrack.us/congress/votes/{congress}-{year}/{h}/export/csv').status_code == 404:
            h_drop.append(f'{h}')
        else:
            # If it is valid, download the associated CSV and save it to the "data" folder
            govt = requests.get(f'https://www.govtrack.us/congress/votes/{congress}-{year}/{h}/export/csv')
            url_content = govt.content
            csv_file = open(f'data/{year}/{h}.csv', 'wb')
            csv_file.write(url_content)
            csv_file.close()
        
    # afterwards, drop all failed legislation requests
    for x in h_drop:
        house[year].remove(x)

Perform the identical task for the senate CSVs.

In [58]:
for year in years:
    # in 2019 and 2020, it was the 116th congress. In 2021 and 2022, it was the 117the congress.
    if year < 2021: congress = 116
    else: congress = 117
        
    # create list of legislations to drop if they do not exist
    s_drop = []

    # Scraping
    for s in senate[year]:
        # first make sure the request is valid. If it is not, drop the legislation from the list.
        if requests.get(f'https://www.govtrack.us/congress/votes/{congress}-{year}/{s}/export/csv').status_code == 404:
            s_drop.append(f'{s}')
        else:
            # If it is valid, download the associated CSV and save it to the "data" folder
            govt = requests.get(f'https://www.govtrack.us/congress/votes/{congress}-{year}/{s}/export/csv')
            url_content = govt.content
            csv_file = open(f'data/{year}/{s}.csv', 'wb')
            csv_file.write(url_content)
            csv_file.close()
    # afterwards, drop all failed legislation requests          
    for x in s_drop:
        senate[year].remove(x)

Now, to check how many total legislations were downloaded for each chamber.

In [17]:
print('House:',len(house[2019]),len(house[2020]),len(house[2021]),len(house[2022]))
print('House Total:',(len(house[2019])+len(house[2020])+len(house[2021])+len(house[2022])))
print('Senate:',len(senate[2019]),len(senate[2020]),len(senate[2021]),len(senate[2022]))
print('Senate Total:',len(senate[2019])+len(senate[2020])+len(senate[2021])+len(senate[2022]))

House: 701 253 449 451
House Total: 1854
Senate: 428 292 528 357
Senate Total: 1605


Now, I'm going to create a function to read in each CSV when called.

In [7]:
def legislations(x,year):
    
    '''
    
    INPUTS: 
    
    x: legislation number in the form of "h5"
    year: the year
    
    OUTPUT:
    
    a dataframe indexed by politician names (with excess information more than just the name) 
    and a single column of each politician's vote for the specific legislation
    
    '''
    #read the CSV
    votes = pd.read_csv(f'data/{year}/{x}.csv')
    votes.reset_index(inplace=True)
    
    # depending on the type of legislation, modifications had to be made to the columns to make sure all were constructed the same
    if len(votes.columns) > 6:
        votes.drop(columns=votes.columns[7:], axis=1, inplace=True)
        votes.drop(columns='index', axis=1, inplace=True)
        
    # name the columns appropriately and drop unnecessary rows and columns
    votes.columns = ['person', 'state', 'district', x+'-'+str(year), 'name', 'party']
    votes.drop(labels=0,axis=0, inplace=True)
    votes.drop(columns=['person','state','district','party'], axis = 1, inplace = True)
    votes.set_index('name',inplace=True)
    
    return(votes)

## Creating the House DataFrame

The next piece of code creates a registration dictionary where each politician is a different key. This code uses regex to extract the name of each politician in order to create the appropriate key.

Starting with the ***House***

In [25]:
# initialize dictionary of house politicians' votes for each politician.
registration = {}

# This y dictionary saves each legislations set of votes.
# In this bit of code I will only be using this dictionary for the indeces of politician names
y = {}

# per year
for year in years:
    
    # for each legislation
    for i,x in enumerate(house[year]):
        
        # create the legislation dataframe
        y[i] = pd.DataFrame(legislations(x, year))

        # add new politicians into the registration dictionary
        
        # grab the politician from the index
        for pol in y[i].index:
            
            # extract the first and last name
            full_name = re.findall("\s[a-zA-Z]+", pol)
            first_name = full_name[0][1:]
            last_name = full_name[-1]
            final_name = first_name+last_name
            
            # if the politician is not yet a key, make him a key
            if final_name not in registration.keys():
                registration[final_name] = {}

This next piece of code attributes each politicain's individual legislation's vote into a complete dataframe of house legislations across all 4 years.

In [26]:
# per year
for year in years:
    
    # for each legislation
    for i,x in enumerate(house[year]):
        
        # call the individual legislation
        y[i] = legislations(x,year)
        
        # name the vote as legislation number (example = "h5")
        vote = f'{y[i].columns[0]}'
        
        # per politician
        for pol in y[i].index:
            
            # extract the first and last name
            full_name = re.findall("\s[a-zA-Z]+", pol)
            first_name = full_name[0][1:]
            last_name = full_name[-1]
            final_name = first_name+last_name
            
            # input the politician's vote into the appropriate cell
            registration[final_name][vote] = y[i].loc[pol,y[i].columns[0]]
            
# create DataFrame
df_house = pd.DataFrame(registration).T

Checking my work:

In [21]:
df_house.head()

,h1-2019,h2-2019,h3-2019,h4-2019,h5-2019,h6-2019,h7-2019,h8-2019,h9-2019,h10-2019,...,h459-2022,h460-2022,h461-2022,h462-2022,h463-2022,h464-2022,h465-2022,h466-2022,h467-2022,h468-2022
Robert Aderholt,Present,McCarthy,Nay,Nay,Yea,Nay,Nay,Yea,No,Yea,...,Nay,Nay,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea
Rob Bishop,Present,McCarthy,Nay,Nay,Yea,Nay,Nay,Yea,No,Yea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sanford Bishop,Present,Pelosi,Yea,Yea,Nay,Yea,Yea,Nay,Aye,Nay,...,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea
Earl Blumenauer,Present,Pelosi,Yea,Yea,Nay,Yea,Yea,Nay,Aye,Nay,...,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea
Kevin Brady,Present,McCarthy,Nay,Nay,Yea,Nay,Nay,Yea,No,Yea,...,Nay,Nay,Yea,Nay,Yea,Yea,Yea,Not Voting,Nay,Yea


In [27]:
df_house.shape

(523, 1854)

Excellent!
Now, save the dataframe as a CSV:

In [28]:
df_house.to_csv('data/The_House.csv')

## Creating the Senate DataFrame

Now perform the the identical task for the ***Senate***

In [22]:
# initialize dictionary of senate politicians' votes for each politician.
registration = {}

# This y dictionary saves each legislations set of votes.
# In this bit of code I will only be using this dictionary for the indeces of politician names
y = {}

# per year
for year in years:
    
    # for each legislation
    for i,x in enumerate(senate[year]):
        
        # create the legislation dataframe
        y[i] = pd.DataFrame(legislations(x, year))

        # add new politicians into the registration dictionary
        
        # grab the politician from the index
        for pol in y[i].index:
            
            # extract the first and last name
            full_name = re.findall("\s[a-zA-Z]+", pol)
            
            # This next if statement only applies here because the Vice President casts tiebreaking votes in the senate
            if "President" not in full_name[0]:
                first_name = full_name[0][1:]
                last_name = full_name[-1]
            else:
                first_name = full_name[1][1:]
                last_name = full_name[-1]                
            final_name = first_name+last_name
            
            # if the politician is not yet a key, make him a key
            if final_name not in registration.keys():
                registration[final_name] = {}

In [23]:
# per year
for year in years:
    
    # for each legislation
    for i,x in enumerate(senate[year]):
        
        # call the individual legislation
        y[i] = legislations(x,year)
        
        # name the vote as legislation number (example = "s205")
        vote = f'{y[i].columns[0]}'
        
        # per politician
        for pol in y[i].index:
            
            # extract the first and last name
            full_name = re.findall("\s[a-zA-Z]+", pol)
            
            # This next if statement only applies here because the Vice President casts tiebreaking votes in the senate
            if "President" not in full_name[0]:
                first_name = full_name[0][1:]
                last_name = full_name[-1]
            else:
                first_name = full_name[1][1:]
                last_name = full_name[-1]                
            final_name = first_name+last_name
            
            # input the politician's vote into the appropriate cell
            registration[final_name][vote] = y[i].loc[pol,y[i].columns[0]]
            
# create DataFrame
df_senate = pd.DataFrame(registration).T

In [29]:
df_senate.shape

(111, 1605)

In [30]:
df_senate.to_csv('data/The_Senate.csv')

**************************

## Politician, Committee, and Trades Data

### Capitol Trades

From Capitol Trades, I will assemble the Politician, Committee, and Trades DataFrames.


First, collect an overall summary of politician trade information.  

This will give the total number of trades, number of filings, total volume of shares traded, number of politicians who traded, and number of issuers since November 2019.

In [2]:
# perform the get request which returns a json file
politician_counts = requests.get('https://bff.capitoltrades.com/trades/digest?metric=countTrades&metric=countFilings&metric=volume&metric=countPoliticians&metric=countIssuers')
politician_counts = politician_counts.json()
total_summary = politician_counts['data']

# gives the total
total_summary

{'countTrades': 45930,
 'countFilings': 2338,
 'volume': 2146923838,
 'countPoliticians': 220,
 'countIssuers': 3639}

Now, I'll scrape for all the politician IDs (and other info).

In [3]:
# The scraping loop for politicians
politician_counts = requests.get('https://bff.capitoltrades.com/trades/digest?metric=countTrades&metric=countFilings&metric=volume&metric=countPoliticians&metric=countIssuers')
politician_counts = politician_counts.json()
p_count_total = politician_counts['data']['countPoliticians'] # total number of politicians

# the scrape can only get 100 trades per page, so solve for total pages
hundreds = math.ceil(p_count_total/100)

#initialize the list
politician_info = []

# scrape the politician information
for i in range(hundreds):
    politicians = requests.get(f'https://bff.capitoltrades.com/politicians?page={i+1}&pageSize=100&metric=dateLastTraded&metric=countTrades&metric=countIssuers&metric=volume').json()
    
    # create a list of all the trades
    for politician in politicians['data']:
        politician_info.append(politician)
        
# Make sure I got them all   
print(len(politician_info))

220


In [4]:
# sample of info gathered from the politician scrape
politician_info[0]

{'_politicianId': 'B001311',
 '_stateId': 'nc',
 'party': 'republican',
 'partyOther': None,
 'district': '9',
 'firstName': 'James',
 'lastName': 'Bishop',
 'nickname': None,
 'middleName': 'Daniel',
 'fullName': 'Bishop, James Daniel',
 'dob': '1964-07-01',
 'gender': 'male',
 'socialFacebook': None,
 'socialTwitter': '@RepDanBishop',
 'socialYoutube': None,
 'website': 'https://danbishop.house.gov/',
 'chamber': 'house',
 'committees': ['hshm', 'hsju'],
 'stats': {'dateLastTraded': '2022-12-01',
  'countTrades': 2,
  'countIssuers': 1,
  'volume': 350000}}

## Creating the Politician DataFrame

First, create a dataframe with all the politicain info (unseparated)

In [5]:
# initialize the DataFrame
politician_df = pd.DataFrame()

# for the number of distinct politicians
for i in range(len(politician_info)):
    
    # Create a dataframe of a single row (1 politician)
    politician_df_sub = pd.DataFrame.from_dict(politician_info[i], orient='index').T
    
    # Then concatenate to the politician_df
    politician_df = pd.concat([politician_df, politician_df_sub], axis=0)
    
# Reset the index
politician_df.reset_index(drop=True,inplace=True)

Check the result

In [6]:
politician_df.shape

(220, 19)

In [7]:
politician_df.head()

,_politicianId,_stateId,party,partyOther,district,firstName,lastName,nickname,middleName,fullName,dob,gender,socialFacebook,socialTwitter,socialYoutube,website,chamber,committees,stats
0,B001311,nc,republican,None,9,James,Bishop,None,Daniel,"Bishop, James Daniel",1964-07-01,male,None,@RepDanBishop,None,https://danbishop.house.gov/,house,"[hshm, hsju]","{'dateLastTraded': '2022-12-01', 'countTrades'..."
1,K000389,ca,democrat,None,17,Rohit,Khanna,Ro,None,"Khanna, Rohit (Ro)",1976-09-13,male,RepRoKhanna,@RepRoKhanna,None,https://khanna.house.gov,house,"[hsag, hsas, hsgo]","{'dateLastTraded': '2022-11-30', 'countTrades'..."
2,B000574,or,democrat,None,3,Earl,Blumenauer,None,Francis,"Blumenauer, Earl Francis",1948-08-16,male,None,@BlumenauerMedia,RepBlumenauer,https://blumenauer.house.gov,house,[hswm],"{'dateLastTraded': '2022-11-30', 'countTrades'..."
3,C000174,de,democrat,None,None,Thomas,Carper,None,Richard,"Carper, Thomas Richard",1947-01-23,male,tomcarper,@SenatorCarper,senatorcarper,https://www.carper.senate.gov/public,senate,"[ssev, ssfi, ssga]","{'dateLastTraded': '2022-11-29', 'countTrades'..."
4,L000559,ri,democrat,None,2,James,Langevin,Jim,R,"Langevin, James R (Jim)",1964-04-22,male,CongressmanJimLangevin,@JimLangevin,jimlangevin,https://langevin.house.gov,house,"[hsas, hshm]","{'dateLastTraded': '2022-11-29', 'countTrades'..."


The last column, stats, contains a dictionary of information per politician.  
I'll extract it.

In [8]:
# Create a stats DataFrame
stats_df = politician_df["stats"].apply(pd.Series)
stats_df.reset_index(inplace=True)
stats_df.drop(['index'],axis=1,inplace=True)

# Chack it out
stats_df.head()

,dateLastTraded,countTrades,countIssuers,volume
0,2022-12-01,2,1,350000
1,2022-11-30,14255,1256,227180000
2,2022-11-30,248,103,5194000
3,2022-11-29,584,219,6454000
4,2022-11-29,273,21,7274500


Looks good! Now combine them.

In [9]:
# Concatenating
politicians_df = pd.concat([politician_df, stats_df], axis=1)

# Also, create the full name column
politicians_df['fullName'] = politicians_df['firstName'] + ' ' + politicians_df['lastName']

# and a full nickname column (this will be needed later to link the different dataframes)
politicians_df['fullnickName'] = politicians_df['nickname'] + ' ' + politicians_df['lastName']

politicians_df.head()

,_politicianId,_stateId,party,partyOther,district,firstName,lastName,nickname,middleName,fullName,...,socialYoutube,website,chamber,committees,stats,dateLastTraded,countTrades,countIssuers,volume,fullnickName
0,B001311,nc,republican,None,9,James,Bishop,None,Daniel,James Bishop,...,None,https://danbishop.house.gov/,house,"[hshm, hsju]","{'dateLastTraded': '2022-12-01', 'countTrades'...",2022-12-01,2,1,350000,NaN
1,K000389,ca,democrat,None,17,Rohit,Khanna,Ro,None,Rohit Khanna,...,None,https://khanna.house.gov,house,"[hsag, hsas, hsgo]","{'dateLastTraded': '2022-11-30', 'countTrades'...",2022-11-30,14255,1256,227180000,Ro Khanna
2,B000574,or,democrat,None,3,Earl,Blumenauer,None,Francis,Earl Blumenauer,...,RepBlumenauer,https://blumenauer.house.gov,house,[hswm],"{'dateLastTraded': '2022-11-30', 'countTrades'...",2022-11-30,248,103,5194000,NaN
3,C000174,de,democrat,None,None,Thomas,Carper,None,Richard,Thomas Carper,...,senatorcarper,https://www.carper.senate.gov/public,senate,"[ssev, ssfi, ssga]","{'dateLastTraded': '2022-11-29', 'countTrades'...",2022-11-29,584,219,6454000,NaN
4,L000559,ri,democrat,None,2,James,Langevin,Jim,R,James Langevin,...,jimlangevin,https://langevin.house.gov,house,"[hsas, hshm]","{'dateLastTraded': '2022-11-29', 'countTrades'...",2022-11-29,273,21,7274500,Jim Langevin


Before I complete the politician dataframe, I will extract the committee information to set up the committee dataframe.

In [10]:
# Initialize dataframe with politician names, chamber, and the committee lists
committee_df = pd.DataFrame(politicians_df[['fullName','chamber','committees']])
committee_df.reset_index(drop=True,inplace=True)

Now, drop the unneccessary columns and save the politician dataframe to a CSV.

In [11]:
politicians_df.drop(['committees','stats'], axis=1, inplace=True)

In [197]:
politicians_df.to_csv('data/Politicians.csv')

## Creating the Committee Dataframe

In [12]:
# check what we have so far
committee_df.head()

,fullName,chamber,committees
0,James Bishop,house,"[hshm, hsju]"
1,Rohit Khanna,house,"[hsag, hsas, hsgo]"
2,Earl Blumenauer,house,[hswm]
3,Thomas Carper,senate,"[ssev, ssfi, ssga]"
4,James Langevin,house,"[hsas, hshm]"


In [13]:
# create a comittee list of all the unique committees
committee_list = []

# for each row's list of committees
for cell in committee_df['committees']:
    # for each committee in the list
    for committee in cell:
        # append the committee
        committee_list.append(committee)
        
# create the distinct committee list
committee_list = np.unique(committee_list)

# create a column for each committee and initialize it to 0
for committee in committee_list:
    committee_df[committee] = 0
    
# set the committees to 1 for the politicains who belong to that committee
for index,committee in enumerate(committee_df['committees']):
    for com in committee:
        committee_df.loc[index,com] = 1

# drop the original column of lists of committees from committee_df
committee_df.drop(['committees'], axis=1, inplace=True)

Check my work

In [14]:
committee_df.head()

,fullName,chamber,hsag,hsap,hsas,hsba,hsbu,hscn,hsed,hsfa,...,sseg,ssev,ssfi,ssfr,ssga,sshr,ssju,ssra,sssb,ssva
0,James Bishop,house,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rohit Khanna,house,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Earl Blumenauer,house,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Thomas Carper,senate,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
4,James Langevin,house,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
committee_df.shape

(220, 51)

Excellent! Save to CSV

In [91]:
committee_df.to_csv('data/committee.csv')

## Creating the Trade DataFrame

First I have to scrape for all the actual trade information.

Create a list of the politicians' IDs as that is what is needed to write the get requests.

In [16]:
# convert the politicianId column to a list of unique IDs
politician_id_list = pd.unique(politicians_df['_politicianId'].tolist())

Scrape the Capitol Trades website of all the information for every trade

In [115]:
# Create an empty list to store the trades
tested = []

# for each politician ID
for politician in politician_id_list:
    
    # The scraping loop for every trade

    # for each politician, first get the total number of trades
    trade_count = requests.get(f'https://bff.capitoltrades.com/trades/digest?metric=countTrades&politician={politician}')
    trade_counts = trade_count.json()
    
    # store the total number of trades for the individual politician
    total_trades = trade_counts['data']['countTrades']

    # the scrape can only get 100 trades per page, so solve for total pages
    hundreds = math.ceil(total_trades/100)
    
    # scraping the actual trade information
    
    # for each page
    for i in range(hundreds):
        
        # grab all the trade info for all the trades on the page
        test_page = requests.get(f'https://bff.capitoltrades.com/trades?page={i+1}&pageSize=100&politician={politician}')
        tested_page = test_page.json()

        # append the list of every individual trade's information
        for item in tested_page['data']:
            tested.append(item)
            
    # for each politician ID, print his total number of trades and the cumulative number of trades scraped
    print(politician,total_trades,len(tested)) # helps for watching progress status

B001311 2 2
K000389 14260 14262
B000574 248 14510
F000450 416 14926
B001248 34 14960
S000344 11 14971
H000273 56 15027
L000564 170 15197
H001086 3559 18756
D000617 200 18956
S001217 117 19073
W000809 1 19074
D000399 62 19136
L000579 685 19821
E000296 76 19897
B001292 817 20714
F000462 434 21148
A000378 157 21305
K000375 85 21390
S000250 84 21474
P000197 82 21556
U000031 79 21635
W000812 22 21657
M001135 525 22182
M001157 6353 28535
T000278 614 29149
H001082 447 29596
F000246 203 29799
L000559 273 30072
C001114 206 30278
G000583 1264 31542
L000590 712 32254
C000174 573 32827
G000563 56 32883
W000797 27 32910
S001180 420 33330
B001236 314 33644
B001277 329 33973
C001120 32 34005
S001201 331 34336
H001074 96 34432
L000566 3 34435
F000459 189 34624
C001047 80 34704
J000020 229 34933
G000590 611 35544
K000392 21 35565
L000397 204 35769
B001267 4 35773
G000577 10 35783
G000061 48 35831
R000612 1 35832
W000804 134 35966
E000298 14 35980
S001203 8 35988
S000168 4 35992
M001180 163 36155
S00119

Next, create a dataframe out of the trade information

In [116]:
# initialize the DataFrame
trade_df = pd.DataFrame()

# for every scraped trade
for i in range(len(tested)):
    
    # create a single row dataframe from each individual trade's information and concatenate to the final dataframe
    trade_df_sub = pd.DataFrame.from_dict(tested[i], orient='index').T
    trade_df = pd.concat([trade_df, trade_df_sub], axis=0)

# Reset the index
trade_df.reset_index(drop=True,inplace=True)

Check the results

In [117]:
trade_df.head()

,_txId,_politicianId,_assetId,_issuerId,pubDate,filingDate,txDate,txType,txTypeExtended,hasCapitalGains,...,value,filingId,filingURL,reportingGap,comment,committees,asset,issuer,politician,labels
0,20003756607,B001311,100012046,435546,2022-12-02T14:05:02Z,2022-12-01,2022-11-28,buy,None,False,...,175000,204344521,https://disclosures-clerk.house.gov/public_dis...,3,Subholding Of: TD Ameritrade brokerage account,"[hshm, hsju]","{'assetType': 'municipal-security', 'assetTick...","{'_stateId': None, 'c2iq': None, 'country': No...","{'_stateId': 'nc', 'chamber': 'house', 'dob': ...",[]
1,20003756608,B001311,100012046,435546,2022-12-02T14:05:02Z,2022-12-01,2022-12-01,buy,None,False,...,175000,204344521,https://disclosures-clerk.house.gov/public_dis...,0,Subholding Of: TD Ameritrade brokerage account,"[hshm, hsju]","{'assetType': 'municipal-security', 'assetTick...","{'_stateId': None, 'c2iq': None, 'country': No...","{'_stateId': 'nc', 'chamber': 'house', 'dob': ...",[]
2,20003757160,K000389,100009820,433770,2022-12-07T14:05:15Z,2022-12-06,2022-11-04,buy,None,False,...,8000,204353493,https://disclosures-clerk.house.gov/public_dis...,32,None,"[hsag, hsas, hsgo]","{'assetType': 'stock', 'assetTicker': 'NVDA:US...","{'_stateId': 'ca', 'c2iq': 'SH9VU1F2', 'countr...","{'_stateId': 'ca', 'chamber': 'house', 'dob': ...",[]
3,20003757161,K000389,100010315,434191,2022-12-07T14:05:15Z,2022-12-06,2022-11-04,buy,None,False,...,8000,204353493,https://disclosures-clerk.house.gov/public_dis...,32,None,"[hsag, hsas, hsgo]","{'assetType': 'stock', 'assetTicker': 'PG:US',...","{'_stateId': 'oh', 'c2iq': 'XWQVU1B7', 'countr...","{'_stateId': 'ca', 'chamber': 'house', 'dob': ...",[]
4,20003757162,K000389,100007277,431721,2022-12-07T14:05:15Z,2022-12-06,2022-11-01,buy,None,False,...,8000,204353493,https://disclosures-clerk.house.gov/public_dis...,35,None,"[hsag, hsas, hsgo]","{'assetType': 'stock', 'assetTicker': 'FRC:US'...","{'_stateId': 'ca', 'c2iq': 'YX7VU1C2', 'countr...","{'_stateId': 'ca', 'chamber': 'house', 'dob': ...",[]


In [119]:
trade_df.shape

(45907, 26)

For columns that display dictionaries, I am going to extract all the information and create a new column for each one.

In [120]:
# asset column
# create asset DataFrame
trade_asset = pd.DataFrame()
trade_asset = trade_df["asset"].apply(pd.Series)

# issuer column
# create issuer DataFrame
trade_issuer = pd.DataFrame()
trade_issuer = trade_df["issuer"].apply(pd.Series)

# politician column
# create politician DataFrame
trade_politician = pd.DataFrame()
trade_politician = trade_df["politician"].apply(pd.Series)

For consistency, I'm going to rename the "instrument" column in the asset DataFrame to "asset_instrument"

In [121]:
trade_asset.rename(columns={'instrument': 'asset_instrument'}, inplace=True)

Check how it looks.

In [122]:
trade_asset.head()

,assetType,assetTicker,asset_instrument
0,municipal-security,None,None
1,municipal-security,None,None
2,stock,NVDA:US,None
3,stock,PG:US,None
4,stock,FRC:US,None


In [126]:
trade_asset.shape

(45907, 3)

Again, for consistency, I'm going to rename the columns of the issuer DataFrame to show they came from the "issuer" dictionary

In [123]:
trade_issuer.rename(columns={'_stateId': 'issuer_state', 'c2iq': 'issuer_c2iq', 'country': 'issuer_country', 'sector': 'issuer_sector'}, inplace=True)

Check how it looks.

In [124]:
trade_issuer.head()

,issuer_state,issuer_c2iq,issuer_country,issuerName,issuerTicker,issuer_sector
0,None,None,None,US TREASURY NOTE,None,None
1,None,None,None,US TREASURY NOTE,None,None
2,ca,SH9VU1F2,us,NVIDIA Corporation,NVDA:US,information-technology
3,oh,XWQVU1B7,us,The Procter & Gamble Co,PG:US,consumer-staples
4,ca,YX7VU1C2,us,First Republic Bank,FRC:US,financials


In [127]:
trade_issuer.shape

(45907, 6)

Check how the trade_politician DataFrame looks. It does not need any modification.

In [125]:
trade_politician.head()

,_stateId,chamber,dob,firstName,gender,lastName,nickname,party
0,nc,house,1964-07-01,James,male,Bishop,None,republican
1,nc,house,1964-07-01,James,male,Bishop,None,republican
2,ca,house,1976-09-13,Rohit,male,Khanna,Ro,democrat
3,ca,house,1976-09-13,Rohit,male,Khanna,Ro,democrat
4,ca,house,1976-09-13,Rohit,male,Khanna,Ro,democrat


In [187]:
trade_politician.shape

(45907, 8)

All three smaller dataframes have the same number of rows, as expected.

Next, I want to extract the labels from their lists and set up a column for each of the labels that each trade could be tagged with.

In [129]:
# Initialize the label list
label_list=[]

# for each trade
for i in range(len(tested)):
    
    # for each label in the trade, add the label to the list
    for label in tested[i]['labels']:
        label_list.append(label)

# print the list of unique labels and its length
label_list = pd.unique(label_list)
print(label_list)
print(len(label_list))

['faang' 'metaverse' 'covid' 'crypto' 'diamondhands' 'whale' 'memestock'
 'spac']
8


In [130]:
# initialize the label dataframe
label_df = pd.DataFrame(trade_df['labels'])

# initialize the columns to 0
for label in label_list:
    label_df[label] = 0
    
# set the labels to 1 for the trades that have that label

# get the index and the label list for each trade
for index,label in enumerate(label_df['labels']):
    
    # for each label in the trade's individual label list
    for lab in label:
        
        # set the column associated with that label to 1
        label_df.loc[index,lab] = 1

# finally, drop the overall labels column
label_df.drop(columns=['labels'], inplace=True)

Check the result.

In [131]:
label_df.head()

,faang,metaverse,covid,crypto,diamondhands,whale,memestock,spac
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0


In [132]:
label_df.shape

(45907, 8)

Proper number of rows, good.

Now, I'll combine all the dataframes into a completed trade DataFrame.

In [133]:
# combining all the trade dataframes
trade_df_complete = pd.DataFrame()
trade_df_complete = pd.concat([trade_df, trade_asset, trade_issuer, trade_politician, label_df], axis=1)

In [134]:
# dropping all the no longer necessary columns
trade_df_complete.drop(columns=['asset','issuer','politician','committees','labels'], inplace=True)

Checking my work

In [135]:
trade_df_complete.head()

,_txId,_politicianId,_assetId,_issuerId,pubDate,filingDate,txDate,txType,txTypeExtended,hasCapitalGains,...,nickname,party,faang,metaverse,covid,crypto,diamondhands,whale,memestock,spac
0,20003756607,B001311,100012046,435546,2022-12-02T14:05:02Z,2022-12-01,2022-11-28,buy,None,False,...,None,republican,0,0,0,0,0,0,0,0
1,20003756608,B001311,100012046,435546,2022-12-02T14:05:02Z,2022-12-01,2022-12-01,buy,None,False,...,None,republican,0,0,0,0,0,0,0,0
2,20003757160,K000389,100009820,433770,2022-12-07T14:05:15Z,2022-12-06,2022-11-04,buy,None,False,...,Ro,democrat,0,0,0,0,0,0,0,0
3,20003757161,K000389,100010315,434191,2022-12-07T14:05:15Z,2022-12-06,2022-11-04,buy,None,False,...,Ro,democrat,0,0,0,0,0,0,0,0
4,20003757162,K000389,100007277,431721,2022-12-07T14:05:15Z,2022-12-06,2022-11-01,buy,None,False,...,Ro,democrat,0,0,0,0,0,0,0,0


In [136]:
# Showing a transpose of the DataFrame so I can inspect the columns 
trade_df_complete.T

,0,1,2,3,4,5,6,7,8,9,...,45897,45898,45899,45900,45901,45902,45903,45904,45905,45906
_txId,20003756607,20003756608,20003757160,20003757161,20003757162,20003757163,20003757164,20003757165,20003757166,20003757167,...,20000523263,20000523264,20000523265,20000523266,20000523267,20000523268,20000523269,20000523270,20000523271,20000523272
_politicianId,B001311,B001311,K000389,K000389,K000389,K000389,K000389,K000389,K000389,K000389,...,L000557,L000557,L000557,L000557,L000557,L000557,L000557,L000557,L000557,L000557
_assetId,100012046,100012046,100009820,100010315,100007277,100018150,100005696,100007277,100005696,100005696,...,100010105,100010474,100011271,100011285,100011542,100011672,100010472,100011950,100012498,100012671
_issuerId,435546,435546,433770,434191,431721,433043,430399,431721,430399,430399,...,434018,434317,434919,434933,435147,435244,434316,435467,435886,436024
pubDate,2022-12-02T14:05:02Z,2022-12-02T14:05:02Z,2022-12-07T14:05:15Z,2022-12-07T14:05:15Z,2022-12-07T14:05:15Z,2022-12-07T14:05:15Z,2022-12-07T14:05:15Z,2022-12-07T14:05:15Z,2022-12-07T14:05:15Z,2022-12-07T14:05:15Z,...,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z,2020-02-02T00:00:00Z
filingDate,2022-12-01,2022-12-01,2022-12-06,2022-12-06,2022-12-06,2022-12-06,2022-12-06,2022-12-06,2022-12-06,2022-12-06,...,2020-01-31,2020-01-31,2020-01-31,2020-01-31,2020-01-31,2020-01-31,2020-01-31,2020-01-31,2020-01-31,2020-01-31
txDate,2022-11-28,2022-12-01,2022-11-04,2022-11-04,2022-11-01,2022-11-15,2022-11-28,2022-11-02,2022-11-07,2022-11-03,...,2019-12-18,2019-12-18,2019-12-18,2019-12-18,2019-12-18,2019-12-18,2019-12-18,2019-12-18,2019-12-18,2019-12-18
txType,buy,buy,buy,buy,buy,buy,buy,buy,buy,buy,...,sell,sell,sell,sell,sell,sell,sell,sell,sell,sell
txTypeExtended,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
hasCapitalGains,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [137]:
trade_df_complete.shape

(45907, 46)

Excellent! Save to CSV.

In [188]:
trade_df_complete.to_csv('data/Trades.csv')

*******************************
## Creating the Stocks DataFrame

The final data wrangling part of the project is to acquire the stock information (daily prices) for each ticker that was traded in the Trades Dataframe.

I'll need three pieces of information from each trade to make the get requests: stock ticker, transaction date, and transaction type (buy or sell).  

First, I'll have to go through the asset tickers and issuer tickers. If both are "NaN", I'll drop the trade.   
With the remainders, if the asset tickers and issuer tickers differ from each other, I'll also drop the trade. (With more time, I would have imputed one from the other or determined which of the 2 to use.) (Approximately, an additional 2000 trades were lost of the remaining 42,000.) 

In [138]:
# Dropping trades for which both ticker columns were "NaN"

# df1 grabs all trades where the issuerTicker is null
df1 = trade_df_complete[(trade_df_complete.issuerTicker.isnull())]

# df2 grabs all trades from df1 where the assetTicker is null
df2 = df1[(df1.assetTicker.isnull())]

In [139]:
# show how many are being dropped
print(len(df2.index))

3287


In [140]:
# Drop them
trade_df_complete.drop(df2.index,inplace=True)

# How many remain?
trade_df_complete.shape[0]

42620

Dropping the trades with differences between asset ticker and issuer ticker

In [141]:
# Dropping the trades
trade_df_complete.drop(trade_df_complete[trade_df_complete['assetTicker'] != trade_df_complete['issuerTicker']].index, inplace = True)

# Resetting the index
trade_df_complete.reset_index(drop=True,inplace=True)

# How many remain?
trade_df_complete.shape[0]

40601

An extra drop of just over 2000 trades.

Now, I have to get the tickers in the right format for the get request.  
Tickers are currently in the form of XXXX:US, but they need to be in the from XXXX.  
Crypto is in the from of $XXX, but similarly needs to be in the form of XXX.  
Some other tickers are in formats with additional non alphabetical characters after the alphabetical ones. XXXX6789 must be in the form XXXX.

In [145]:
# Creating a ticker dataframe
ticker_df = pd.DataFrame(trade_df_complete['assetTicker'])

# Let's see what they currently look like
ticker_df.head()

,assetTicker
0,NVDA:US
1,PG:US
2,FRC:US
3,KMX:US
4,FRC:US


Using RegEx, I'll transform the tickers to proper format.

In [146]:
# Initialize a ticker list
tickers = []

# Set a counter
counter=0

# for each ticker
for ticker in trade_df_complete['assetTicker']:
    
    # look for the ":" and if it exists
    if len(re.findall(".+:", ticker)) == 1:
        
        # transform into the format of everything preceding the ":"
        tick = re.findall(".+:", ticker)  
        tick = tick[0][0:-1]
        
    # look for the "$" which indicates crypto and if it exists    
    elif len(re.findall("^[$].+", ticker)) == 1:

        # transform into the format of everything after the "$"
        tick = re.findall("^[$].+", ticker)
        tick = tick[0][1:]
    
    # If non-alphabetical characters exist
    elif len(re.findall("[A-Z]+", ticker)) == 1:
        
        # transform into the proper format
        tick = re.findall("[A-Z]+", ticker)
        tick = tick[0]
    # If it is something else entirely, skip it
    else:
        break
    
    # set the ticker to its proper format within the ticker dataframe
    ticker_df.loc[counter] = tick
    
    # Create the unique set of tickers
    if tick not in tickers:
        tickers.append(tick) 
        
    counter += 1

Check the length. It should match the trade_df_complete length.

In [147]:
ticker_df.shape[0]

40601

Confirm that the transformation worked.

In [148]:
ticker_df.head()

,assetTicker
0,NVDA
1,PG
2,FRC
3,KMX
4,FRC


Create a column of the newly formatted tickers. 

In [150]:
trade_df_complete['newTicker'] = ticker_df['assetTicker']

Next, I'll create a dataframe with just the variables I need for the get request.

In [151]:
Transactions_df = trade_df_complete[['newTicker','txDate','txType']]

In [152]:
Transactions_df.head()

,newTicker,txDate,txType
0,NVDA,2022-11-04,buy
1,PG,2022-11-04,buy
2,FRC,2022-11-01,buy
3,KMX,2022-11-28,buy
4,FRC,2022-11-02,buy


In order to find the earliest transaction date for each ticker, I'll create a dataframe that groups by ticker to find the earliest date.

In [155]:
# Create the timeframe dataframe
timeframe = Transactions_df.groupby(['newTicker'])
timeframe = timeframe.agg(first_date=('txDate', np.min)).reset_index()

In [156]:
#display it
display(timeframe)

,newTicker,first_date
0,1QY,2021-02-18
1,667517Q,2019-12-09
2,A,2020-01-08
3,AA,2020-04-13
4,AAGIY,2020-01-03
...,...,...
2501,ZS,2020-12-11
2502,ZTS,2019-12-12
2503,ZUO,2019-12-13
2504,ZURVY,2022-03-16


There are 2,506 unique tickers for which I now have to iterate through to pull the stock information. I'll start each pull from 1 week before the transaction was made ana end it on Nov 28, 2022.

In [157]:
# Creating lists to iterate through

# symbol list
symbols = timeframe['newTicker'].tolist()

# start date list
starts = timeframe['first_date'].tolist()

I have to convert the start dates to the proper type so I can do the appropriate date math.

In [163]:
# Initialize new start date list
starting = []

# for each start date
for start in starts:
    
    # convert
    d1 = datetime.strptime(start, "%Y-%m-%d")
    
    # Subtract 7 days
    d2 = d1 - timedelta(days=7)
    
    # grab the date part of the new start time
    starting.append(str(d2)[0:10])
    
    # the end date for all tickers
    end = datetime(2022,11,28)

Now, I'm ready for the ticker scraping.

In [165]:
# The get request variables to grab the info from yahoo finance
url="https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv"
s = requests.get(url).content
companies = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [172]:
# initialize the DataFrame
stock_final = pd.DataFrame()

# initiate counter
counter = 0

# iterate over each symbol
for sym in symbols:  
    
    # print the symbol which is being downloaded
    print( str(symbols.index(sym)) + str(' : ') + sym, sep=',', end=',', flush=True)  
    
    # add try/except to keep going in case any ticker fails to produce stock information for whatever reason
    try:
        # download the stock price 
        stock = []
        stock = yf.download(sym,start=starting[counter], end=end, progress=False, actions=True)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=sym
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
    # update counter
    counter+=1

0 : 1QY,
1 Failed download:
- 1QY: No data found for this date range, symbol may be delisted
1 : 667517Q,
1 Failed download:
- 667517Q: No data found for this date range, symbol may be delisted
2 : A,3 : AA,

C:\Users\hamao\AppData\Local\Temp\ipykernel_548\2651192191.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock,sort=False)


4 : AAGIY,

C:\Users\hamao\AppData\Local\Temp\ipykernel_548\2651192191.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock,sort=False)


5 : AAL,

...

C:\Users\hamao\AppData\Local\Temp\ipykernel_548\2651192191.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock,sort=False)


2504 : ZURVY,

C:\Users\hamao\AppData\Local\Temp\ipykernel_548\2651192191.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock,sort=False)


2505 : ZWS,

C:\Users\hamao\AppData\Local\Temp\ipykernel_548\2651192191.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock,sort=False)
C:\Users\hamao\AppData\Local\Temp\ipykernel_548\2651192191.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock,sort=False)


Let's see what the DataFrame looks like.

In [173]:
stock_final.head()

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Name
Date,,,,,,,,,
2020-01-02,85.900002,86.349998,85.199997,85.949997,84.371025,1410500.0,0.0,0.0,A
2020-01-03,84.669998,85.330002,84.500000,84.570000,83.016365,1118300.0,0.0,0.0,A
2020-01-06,84.000000,84.820000,83.599998,84.820000,83.261772,1993200.0,0.0,0.0,A
2020-01-07,83.959999,85.260002,83.940002,85.080002,83.516998,1684700.0,0.0,0.0,A
2020-01-08,85.959999,86.470001,85.199997,85.919998,84.341576,1847600.0,0.0,0.0,A


In [174]:
stock_final.shape

(1349362, 9)

Determine how many tickers for which I found stock data.

In [175]:
all_ticks = stock_final['Name'].tolist()
len(pd.unique(all_ticks))

2298

Save this info to a CSV.

In [177]:
stock_final.to_csv('data/stock_final.csv')

## This concludes the data wrangling portion of the project. 

### Please proceed to notebook 2.